In [28]:
import pandas as pd
import os
import numpy as np
import math
from datetime import datetime, date

In [2]:
import sys
sys.path.append('.//..//')
print(sys.path)
%load_ext autoreload
%autoreload 2

['H:\\github\\felippetadeu\\analise_fundos_imobiliarios\\notebooks', 'C:\\Users\\felip\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\felip\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\felip\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\felip\\AppData\\Local\\Programs\\Python\\Python311', 'H:\\github\\felippetadeu\\analise_fundos_imobiliarios\\env', '', 'H:\\github\\felippetadeu\\analise_fundos_imobiliarios\\env\\Lib\\site-packages', 'H:\\github\\felippetadeu\\analise_fundos_imobiliarios\\env\\Lib\\site-packages\\win32', 'H:\\github\\felippetadeu\\analise_fundos_imobiliarios\\env\\Lib\\site-packages\\win32\\lib', 'H:\\github\\felippetadeu\\analise_fundos_imobiliarios\\env\\Lib\\site-packages\\Pythonwin', './/..//']


In [4]:
from src.application.services.fii_service import FIIService
service = FIIService()
df = service.start()

could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'
could not convert string to float: 'N/A'


In [5]:
df

,simbolo,tipo,sub_tipo,descricao,dividend_yield,patrimonio_liquido,liquidez_diaria_media,ultimo_rendimento,valor_patrimonial,rentabilidade_mes,p_vp,cotas_emitidas,numero_cotistas,min_semanas,max_semanas,valorizacao
0,AAZQ11,Fiagro,Indefinido,None,16.87,232000000.0,"1,1M",0.93,9.65,2.14,None,24037284,21961,8.49,9.90,5.72
1,ABCP11,Tijolo,Shoppings,None,8.69,432700000.0,"75,7K",0.81,91.88,4.10,None,4709082,17274,63.99,74.20,5.66
2,AFHI11,Papel,Papéis,None,12.60,322400000.0,"866,3K",1.04,96.43,-0.60,None,3343095,26665,92.17,100.98,3.67
3,AIEC11,Tijolo,Lajes Corporativas,None,16.47,459200000.0,"868,7K",0.59,95.16,-3.01,None,4824987,16416,53.00,72.86,16.82
4,ALZM11,Fundo de Fundos,Fundo de Fundos,None,117.61,132700000.0,"235,4K",0.10,87.42,-0.45,None,1518054,5416,7.94,9.57,0.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,XPPR11,Fundo Misto,Lajes Corporativas,None,6.31,508300000.0,"426,4K",0.32,69.48,-0.67,None,7316171,55515,18.01,34.42,35.62
368,XPSF11,Papel,Fundo de Fundos,None,10.90,366900000.0,"643,4K",0.98,8.47,1.09,None,43302140,59102,6.67,8.75,15.86
369,YUFI11,Fundo Misto,Fundo de Desenvolvimento,None,2.89,50500000.0,351,0.86,102.44,-0.01,None,492650,60,63.01,105.00,7.57
370,ZAVI11,Outros,Indefinido,None,10.49,158400000.0,"328,0K",0.96,134.97,1.76,None,1173322,4790,120.00,136.00,4.98


In [11]:
def pontuacao_fii(row):
    # Atribua uma pontuação com base nos critérios fornecidos.
    pontuacao = 0
    
    # Critério 1: Dividend Yield > 9
    if row['dividend_yield'] > 9:
        pontuacao += 1
    
    # Critério 2: Patrimônio Líquido > 300000 e contém a letra 'M'
    if row['patrimonio_liquido'] > 300000000:
        pontuacao += 1
    
    # Critério 3: Valorização > -9
    if row['valorizacao'] > -9:
        pontuacao += 1
    
    # Critério 4: Diferença entre Min_Semanas e Max_Semanas seja pequena
    if abs(row['min_semanas'] - row['max_semanas']) < 5:
        pontuacao += 1
    else:
        pontuacao -= 1
    
    # Critério 5: Proporção de cotas_emitidas e numero_cotistas seja alta
    if (row['cotas_emitidas']/row['numero_cotistas']) > 176 and row['numero_cotistas'] > 10000:
        pontuacao += 1
    
    # Critério 6: Rentabilidade Mensal seja positiva
    if row['rentabilidade_mes'] > 0:
        pontuacao += 1
    
    return pontuacao


In [29]:
def minimo_gerar_cota(row):
    # Atribua uma pontuação com base nos critérios fornecidos.
    valor = 0
    ultimo_rendimento = row['ultimo_rendimento']
    valor_patrimonial = row['valor_patrimonial']
    if ultimo_rendimento > 0:
        valor = (valor_patrimonial/ultimo_rendimento) * valor_patrimonial
    
    return math.ceil(valor)
    
def minimo_gerar_100(row):
    # Atribua uma pontuação com base nos critérios fornecidos.
    valor = 0
    ultimo_rendimento = row['ultimo_rendimento']
    valor_patrimonial = row['valor_patrimonial']
    if ultimo_rendimento > 0:
        valor = (100/ultimo_rendimento) * valor_patrimonial
    
    return math.ceil(valor)


In [30]:
# Calcule a pontuação para cada fundo imobiliário.
df['Pontuacao'] = df.apply(pontuacao_fii, axis=1)
df['Mín. Gerar Cota'] = df.apply(minimo_gerar_cota, axis=1)
df['Mín. Gerar R$100'] = df.apply(minimo_gerar_100, axis=1)

# Ordene os fundos imobiliários com base na pontuação.
fundos_ordenados = df.sort_values(by='Pontuacao', ascending=False)

In [31]:
fundos_ordenados[fundos_ordenados['Pontuacao'] >= 5]

,simbolo,tipo,sub_tipo,descricao,dividend_yield,patrimonio_liquido,liquidez_diaria_media,ultimo_rendimento,valor_patrimonial,rentabilidade_mes,p_vp,cotas_emitidas,numero_cotistas,min_semanas,max_semanas,valorizacao,Pontuacao,Mín. Gerar Cota,Mín. Gerar R$100
293,RZAG11,Fiagro,Indefinido,None,15.92,6.635000e+08,"2,1M",0.95,9.75,0.54,None,68040425,83664,8.69,9.87,1.59,6,101,1027
340,VGIR11,Papel,Papéis,None,14.63,1.000000e+09,"4,0M",1.02,9.76,0.61,None,103220707,230050,9.12,10.01,0.10,6,94,957
206,MCHF11,Papel,Papéis,None,12.22,3.106000e+08,"1,3M",0.96,9.60,2.11,None,32354247,84727,8.07,9.42,0.75,6,96,1000
218,MXRF11,Papel,Papéis,None,12.98,2.600000e+09,"12,6M",1.08,9.86,1.14,None,259279007,999290,10.00,11.12,5.14,6,91,913
306,SNAG11,Fiagro,Papéis,None,13.60,5.044000e+08,"1,6M",1.00,10.09,0.10,None,50000000,73275,10.00,10.40,0.49,6,102,1009
69,CPTS11,Papel,Papéis,None,10.17,2.900000e+09,"6,1M",0.95,9.04,0.58,None,317828140,295246,7.49,9.04,5.37,6,87,952
338,VGIA11,Fiagro,Indefinido,None,17.28,8.327000e+08,"3,9M",0.94,9.63,1.45,None,86455117,161613,8.03,10.05,9.21,6,99,1025
190,KNSC11,Papel,Papéis,None,11.09,1.200000e+09,"3,1M",1.02,9.06,0.77,None,132757940,122671,8.45,9.21,7.01,6,81,889
343,VINO11,Tijolo,Lajes Corporativas,None,9.51,8.983000e+08,"1,1M",0.71,10.85,0.79,None,82826295,144500,7.19,9.50,18.83,6,166,1529
368,XPSF11,Papel,Fundo de Fundos,None,10.90,3.669000e+08,"643,4K",0.98,8.47,1.09,None,43302140,59102,6.67,8.75,15.86,6,74,865
